In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
from datetime import date
import pandas as pd
import requests
import time

In [2]:
chromeDriver = "driver/chromedriver"

In [3]:
today = date.today()
anoAtual = today.strftime("%Y")
mesAtual = today.strftime("%m")

In [4]:
def getDeputadosList():
    # Definições do Selenium,
    driver = webdriver.Chrome(chromeDriver)
    driver.get("https://www.camara.leg.br/deputados/quem-sao")

    # Definições do BeaultifulSoup.
    page = requests.get(driver.current_url)
    soup = BeautifulSoup(page.text, 'html.parser')
    
    # Corre select box na página pegando o código e o nome dos deputados.
    lista_deputados = soup.find(id='parametro-nome').findAll('option')
    listDeputadosCode = []
    for item in lista_deputados:
        if item.get('value'):
            listDeputadosCode.append(
                {
                    'codigo': item.get('value'),
                    'nome_deputado': item.getText()
                }
            )
    dfListaDepCode = pd.DataFrame(listDeputadosCode)
    driver.close()
    return dfListaDepCode
dfListaDepCode = getDeputadosList()

In [5]:
dfListaDepCode

,codigo,nome_deputado
0,204554,Abílio Santana
1,204521,Abou Anni
2,204379,Acácio Favacho
3,204560,Adolfo Viana
4,204528,Adriana Ventura
...,...,...
565,178889,Zé Carlos
566,204559,Zé Neto
567,160632,Zé Silva
568,204517,Zé Vitor


### Pegando informações de gastos e recursos

In [6]:
dicGastosRecursos = {}

In [7]:
# Parametrização
# depCode = '204517'
# ano = anoAtual

In [8]:
def getAnosMandato(depCode):
    url = 'https://www.camara.leg.br/deputados/'+depCode
    driver = webdriver.Chrome(chromeDriver)
    driver.get(url)

    page = requests.get(driver.current_url)
    soup = BeautifulSoup(page.text, 'html.parser')

    # Cria uma lista dos anos de mandato do deputado
    anosMandato = []
    srcAnosMandato = soup.find(class_='linha-tempo').findAll(class_='linha-tempo__marco')
    for item in srcAnosMandato:
        anosMandato.append(item.getText())

    driver.close()
    return anosMandato

In [9]:
# Entrano detalhe dos gastos e recursos
# ano = '2021'

In [10]:
GastosRecursos = []
def getGastosRecursos(depCode, ano):
    url = 'https://www.camara.leg.br/deputados/'+depCode+'?ano='+ano+''
    driver = webdriver.Chrome(chromeDriver)
    driver.get(url)

    driver.find_element_by_xpath('/html/body/div[2]/div[1]/main/div[3]/div/div/div[1]/div/section[5]/ul/li[1]/div[2]/a').click()

    page = requests.get(driver.current_url)
    soup = BeautifulSoup(page.text, 'html.parser')

    mesesDetalhe = soup.findAll(class_='detalhe')
    for item in mesesDetalhe:
        dicGastosRecursos = {}

        mesGasto = item.find('a').getText().lstrip().rstrip().split('/')[0]

        dicGastosRecursos['cod_deputado'] = depCode
        dicGastosRecursos['ano'] = ano
        dicGastosRecursos['mes'] = mesGasto

        mesGastoId = item.find('a').get('id')

        driver.find_element_by_xpath('//*[@id="'+mesGastoId+'"]').click()
        time.sleep(3)

        page = requests.get(driver.current_url)
        soup = BeautifulSoup(page.text, 'html.parser')

        linhas = soup.find(class_='tabela-2').findAll('tr')
        #print(len(linhas))
        idValor = 0
        for linha in linhas:
            #print(len(linhas) - 1)
            if linha.findAll('a'):
                dicGastosRecursos['tipo_despesa'] = linha.findAll('a')[0].getText().lstrip().rstrip()
            if idValor < len(linhas):
                if linha.findAll(id='sumarizado'+str(idValor)):
                    dicGastosRecursos['valor_despesa'] = linha.findAll(id='sumarizado'+str(idValor))[0].getText().lstrip().rstrip()
                    idValor = idValor + 1
                    GastosRecursos.append(dicGastosRecursos)

        driver.back()
        time.sleep(2)
        
    driver.close()

In [11]:
depCode = '204517'
anosMandato = getAnosMandato(depCode)
for ano in anosMandato:
    getGastosRecursos(depCode, ano)

dfGastosRecursos = pd.DataFrame(GastosRecursos)
dfGastosRecursos

,cod_deputado,ano,mes,tipo_despesa,valor_despesa
0,204517,2019,Fevereiro,PASSAGEM AÉREA - RPA,"R$ 1.638,52"
1,204517,2019,Fevereiro,PASSAGEM AÉREA - RPA,"R$ 1.638,52"
2,204517,2019,Fevereiro,PASSAGEM AÉREA - RPA,"R$ 1.638,52"
3,204517,2019,Fevereiro,PASSAGEM AÉREA - RPA,"R$ 1.638,52"
4,204517,2019,Fevereiro,PASSAGEM AÉREA - RPA,"R$ 1.638,52"
...,...,...,...,...,...
168,204517,2021,Março,DIVULGAÇÃO DA ATIVIDADE PARLAMENTAR.,"R$ 10.340,00"
169,204517,2021,Março,DIVULGAÇÃO DA ATIVIDADE PARLAMENTAR.,"R$ 10.340,00"
170,204517,2021,Março,DIVULGAÇÃO DA ATIVIDADE PARLAMENTAR.,"R$ 10.340,00"
171,204517,2021,Abril,COMBUSTÍVEIS E LUBRIFICANTES.,"R$ 1.078,08"
